In [14]:
!python --version

Python 3.11.3


In [15]:
from openai import OpenAI

In [16]:
with open("data/openai/openai.env") as f:
    key = f.read()

In [17]:
with open("data/openai/quelle_est_bleue.txt") as f:
    text = f.read()

# print(text)

client = OpenAI(api_key=key)

completion = client.chat.completions.create(model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "De quoi parle ce texte? dans un contexte de plongeur pro"},
        {"role": "user", "content": text}
    ]
)

res = completion.choices[0].message
print(res.content)

Ce texte parle de la mer Méditerranée, de ses eaux bleues et de la vie marine qu'elle abrite. Il évoque la diversité des poissons, des marins et des jolies filles qui fréquentent ses rivages, notamment à Marseille. On y parle également de la pêche, tant professionnelle qu'amateur, et des plaisirs de la chasse sous-marine. Le texte décrit aussi les lieux emblématiques où les gens se retrouvent pour profiter du soleil et de la mer, tels que les plages du Frioul, Catalans et l'Abri-Côtier, ainsi que Callelongue. Enfin, il met en avant le désir de préserver la beauté naturelle de la côte et de lutter contre le bétonnage.


In [18]:
with open("data/openai/churn_cleaning.py") as f:
    text = f.read()

print(text)

client = OpenAI(api_key=key)

completion = client.chat.completions.create(model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Expliques moi ce que fais ce code"},
        {"role": "user", "content": text}
    ]
)

res = completion.choices[0].message
print(res.content)

import pandas as pd
import sweetviz
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

df = pd.read_csv('data/churn/churn.csv', delimiter=',')
print(df.shape)

# Anonymisation
df = df.drop(["RowNumber", "CustomerId", "Surname"], axis = 1)

# France only
df = df[df.Geography == "France"]
df = df.drop(["Geography"], axis = 1)
print(df.describe().T)

# my_report = sweetviz.analyze(df)
# my_report.show_html()

# Discretisation?
print(df.nunique())

# Etude pragmatique sur valeurs discretes (categorielles)
fig, axarr = plt.subplots(2, 2)
sns.countplot(x='Gender', hue = 'Exited',data = df, ax=axarr[0][0])
sns.countplot(x='HasCrCard', hue = 'Exited',data = df, ax=axarr[0][1])
sns.countplot(x='IsActiveMember', hue = 'Exited',data = df, ax=axarr[1][0])
plt.show()

# Etude pragmatique sur valeurs continues
fig, axarr = plt.subplots(3, 2)
sns.boxplot(y='CreditScore',x = 'Exited', hue = 'Exited',data = df, ax=axarr[0][0])
sns.boxplot(y='Age',x = 'Exited',

In [19]:
import datetime
# import whisper # pip install openai-whisper
import os

In [20]:
class OpenAIService:

    def __init__(self, whisper_model="base", chat_model="gpt-3.5-turbo-1106", chat_limit=16385):
        # os.environ["path"] += ";c:\\ffmpeg\\bin"
        # self.model = whisper.load_model(whisper_model)
        with open("data/openai/openai.env") as f:
            key = f.read()
        self.client = OpenAI(api_key=key)
        self.chat_model = chat_model
        self.chat_limit = chat_limit

    def mp3_to_text(self, path):
        result = self.model.transcribe(path)
        return result["text"].encode("utf-8").decode()

    def summary(self, text: str, nb: int=5) -> str:
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": f"Fais moi un résumé en {nb} parties de ce texte"},
                {"role": "user", "content": text}])
        return completion.choices[0].message.content

    def correction(self, text: str, ponctuation=True, bank=True) -> str:
        s = "Corriges moi ce texte "
        if ponctuation:
            s+="avec de la ponctuation "
        if bank:
            s+="avec un vocabulaire bancaire"
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": s},
                      {"role": "user", "content": text}])
        return completion.choices[0].message.content

    def translate(self, text: str, langue="anglais") -> str:
        s = f"Traduis moi ce texte en {langue}"
        completion = self.client.chat.completions.create(
            model=self.chat_model,
            messages=[{"role": "system", "content": s},
                      {"role": "user", "content": text}])
        return completion.choices[0].message.content

In [21]:
print("OpenAI test")
openai = OpenAIService(whisper_model="base")
time0 = datetime.datetime.now()

# res = openai.mp3_to_text("data/openai/bank.mp3")
# print(res.strip())
# with open("data/openai/bank.txt", "w") as f:
#     f.write(res)

with open("data/openai/bank.txt", "r", encoding="iso-8859-1") as f:
    res = f.read()
correction = openai.correction(res, True, True)
print(correction)
with open(f"data/openai/correction.txt", "w") as f:
    f.write(correction)

OpenAI test
Je suis Salimage, j'ai 32 ans, et je suis actuellement conseiller clientèle au sein de la Banque Postale. Après un BTS commercial et une année de spécialisation en école de commerce, j’ai entamé un contrat de qualification d'une durée de 9 mois, dispensé par La Poste et un centre de formation. À l'issue de cette formation, j'ai obtenu un certificat de compétences professionnelles de conseiller financier, ce qui m'a permis aussitôt d'intégrer le poste de conseiller financier dans un bureau de poste pendant trois ans. J'ai ensuite évolué vers un poste de conseiller clientèle que j'occupe depuis 5 ans. Je travaille au nom et pour le compte de la Banque Postale, proposant ses produits et services financiers. J'ai à ma charge 1600 clients, dont les 500 premiers constituent l'essentiel de mon portefeuille. En qualité de conseiller bancaire, ma mission principale est d'apporter des conseils adaptés à mes clients pour optimiser leur épargne et constituer leur patrimoine. Il est imp

In [22]:
summary = openai.summary(res)
print(summary)
with open(f"data/openai/summary5.txt", "w") as f:
    f.write(summary)

1. Parcours professionnel : Salimage est actuellement conseiller clientèle à la Banque Postale, après un BTS commercial, une année de spécialisation en école de commerce et une formation dispensée par La Poste.
2. Mission et activités : Son rôle est d'apporter des conseils financiers adaptés à ses clients, d'assurer une relation de qualité et de promouvoir les produits financiers de la Banque Postale.
3. Gestion de portefeuille : Salimage gère 1600 clients, dont les 500 premiers sont au cœur de son portefeuille, et cherche à développer sa clientèle par des actions de fidélisation et d'approche familiale.
4. Préparation et déroulement des journées : Il consacre du temps chaque matin à la préparation des entretiens pour ses 6-7 rendez-vous quotidiens, suivi de la gestion des appels clients et de l'actualité économique.
5. Enrichissement professionnel : Salimage apprécie le contact avec la clientèle et la diversité des problématiques rencontrées, soulignant l'importance de la dimension co

In [23]:
summary_english = openai.translate(summary)
print(summary_english)
with open(f"data/openai/summary_english.txt", "w") as f:
    f.write(summary_english)

1. Professional Background: Salimage is currently a customer advisor at La Banque Postale, following a BTS in sales, a year of specialization in a business school, and training provided by La Poste.
2. Mission and Activities: His role is to provide tailored financial advice to his clients, ensure a high-quality relationship, and promote La Banque Postale's financial products.
3. Portfolio Management: Salimage manages 1600 clients, with the top 500 being at the core of his portfolio, and seeks to expand his client base through loyalty initiatives and a family-oriented approach.
4. Daily Preparation and Activities: He dedicates time each morning to preparing for his 6-7 daily appointments, followed by managing client calls and staying updated on economic news.
5. Professional Enrichment: Salimage enjoys interacting with clients and the diversity of issues encountered, emphasizing the importance of advisory expertise in this field, and advising young graduates to be assertive and attentiv

In [24]:
summary_jpn = openai.translate(summary, "japonais")
print(summary_jpn)
with open(f"data/openai/summary_jpn.txt", "wb") as f:
    f.write(summary_jpn.encode("utf-8"))

1.職歴：Salimageは現在、ポスト銀行で顧客アドバイザーを務めており、コマーシャルのBTSを修了した後、経済学校での専門課程を1年経て、ポスト局による研修を受けています。
2.ミッションと活動：彼の役割は、顧客に適した財務アドバイスを提供し、質の高い関係を築き、ポスト銀行の金融商品を宣伝することです。
3.ポートフォリオの管理：Salimageは1600人の顧客を管理し、そのうち最初の500人が彼のポートフォリオの中心であり、リテンションとファミリー向けアプローチの活動によって顧客を拡大しようとしています。
4.日々の準備と遂行：彼は毎朝、1日に6〜7回の面談の準備に時間を割き、その後、顧客への電話対応と経済情報の管理に時間を割いています。
5.職務の充実：Salimageは顧客とのコンタクトを楽しんでおり、遭遇する多様な問題に魅力を感じており、この仕事におけるアドバイスの重要性を強調し、若手の卒業生に積極的であり、耳を傾けることを勧めています。
